In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
import calendar
import os
os.chdir('H:\W-GITSYNCH')


%matplotlib inline
sns.set(style="whitegrid")

In [2]:
vix = pd.read_csv("^VIX.csv", index_col="Date")
vix.index = pd.to_datetime(vix.index, format="%Y-%m-%d")
vix = vix.apply(pd.to_numeric, errors="coerce")

spx = pd.read_csv("^GSPC.csv", index_col="Date")
spx.index = pd.to_datetime(spx.index, format="%Y-%m-%d")
spx = spx.apply(pd.to_numeric, errors="coerce")

def DatesAndPct(df):
    df["day"] = df.index.dayofyear
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["pct"] = np.log(df["Adj Close"]).diff(1)
    return df

vix = DatesAndPct(vix)
spx = DatesAndPct(spx)

In [3]:
spx_vix = pd.concat([vix['Adj Close'], spx['Adj Close']], axis = 1)
spx_vix.columns = ['vix','spx']

def DatesAndPct1(df):
    df["day"] = df.index.dayofyear
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["pct vix"] = np.log(df["vix"]).diff(1)
    df["pct spx"] = np.log(df["spx"]).diff(2)
    return df

final_vix_spx = DatesAndPct1(spx_vix).dropna()
final_vix_spx.to_csv('final_vix_spx.csv')

In [15]:
def worst_return(vixlevel,dte):
    df = final_vix_spx.copy()
    df["pct spx dte"] = np.log(df["spx"]).diff(dte)
    df = df[df['vix'] < vixlevel]#.dropna()
    df = df.sort_values(['pct spx dte'], ascending = 1)
    return df

In [16]:
#2 days is 3 data points 
worst_return(12,2).head(10)

,vix,spx,day,month,year,pct vix,pct spx,pct spx dte
Date,,,,,,,,
1995-06-26,11.80,544.130005,177,6,1995,0.059354,-0.012674,-0.012674
2007-01-26,11.13,1422.180054,26,1,2007,-0.008054,-0.012542,-0.012542
2014-07-08,11.98,1963.709961,189,7,2014,0.055785,-0.011005,-0.011005
1993-07-02,11.33,445.839996,183,7,1993,-0.015762,-0.010465,-0.010465
2007-02-12,11.61,1433.369995,43,2,2007,0.044922,-0.010369,-0.010369
2006-11-28,11.62,1386.719971,332,11,2006,-0.056872,-0.010209,-0.010209
1993-07-16,10.96,445.750000,197,7,1993,0.033398,-0.009667,-0.009667
2004-12-17,11.95,1194.199951,352,12,2004,-0.026426,-0.009600,-0.009600
1994-09-02,11.40,470.989990,245,9,1994,-0.039558,-0.009509,-0.009509
